<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.4.1


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [9]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 9s - loss: 0.6879 - accuracy: 0.4988 - val_loss: 0.6923 - val_accuracy: 0.4341

Epoch 00001: saving model to weights.01-0.69.h5
Epoch 2/5
26/26 - 8s - loss: 0.6701 - accuracy: 0.5365 - val_loss: 0.6799 - val_accuracy: 0.4390

Epoch 00002: saving model to weights.02-0.68.h5
Epoch 3/5
26/26 - 8s - loss: 0.6490 - accuracy: 0.5998 - val_loss: 0.6548 - val_accuracy: 0.5122

Epoch 00003: saving model to weights.03-0.65.h5
Epoch 4/5
26/26 - 7s - loss: 0.6135 - accuracy: 0.7287 - val_loss: 0.5941 - val_accuracy: 0.8439

Epoch 00004: saving model to weights.04-0.59.h5
Epoch 5/5
26/26 - 8s - loss: 0.5566 - accuracy: 0.8139 - val_loss: 0.5579 - val_accuracy: 0.7366

Epoch 00005: saving model to weights.05-0.56.h5


In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model-{epoch:02d}-{val_accuracy:.2f}.h5', verbose=1)
          ])

Epoch 1/10
26/26 - 8s - loss: 0.6640 - accuracy: 0.5876 - val_loss: 0.6374 - val_accuracy: 0.7268

Epoch 00001: saving model to model-01-0.73.h5
Epoch 2/10
26/26 - 7s - loss: 0.6219 - accuracy: 0.6752 - val_loss: 0.6422 - val_accuracy: 0.5415

Epoch 00002: saving model to model-02-0.54.h5
Epoch 3/10
26/26 - 7s - loss: 0.5826 - accuracy: 0.7141 - val_loss: 0.5376 - val_accuracy: 0.7805

Epoch 00003: saving model to model-03-0.78.h5
Epoch 4/10
26/26 - 7s - loss: 0.5289 - accuracy: 0.7494 - val_loss: 0.5139 - val_accuracy: 0.6976

Epoch 00004: saving model to model-04-0.70.h5
Epoch 5/10
26/26 - 7s - loss: 0.4651 - accuracy: 0.8005 - val_loss: 0.4286 - val_accuracy: 0.8049

Epoch 00005: saving model to model-05-0.80.h5
Epoch 6/10
26/26 - 7s - loss: 0.3961 - accuracy: 0.8431 - val_loss: 0.4911 - val_accuracy: 0.7463

Epoch 00006: saving model to model-06-0.75.h5
Epoch 7/10
26/26 - 8s - loss: 0.3616 - accuracy: 0.8589 - val_loss: 0.3060 - val_accuracy: 0.8829

Epoch 00007: saving model to mo

## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.002,
              mode='max',
              monitor='val_accuracy',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 7s - loss: 0.6592 - accuracy: 0.6363 - val_loss: 0.6507 - val_accuracy: 0.5463
Epoch 2/50
26/26 - 7s - loss: 0.6192 - accuracy: 0.6509 - val_loss: 0.5562 - val_accuracy: 0.8927
Epoch 3/50
26/26 - 7s - loss: 0.5484 - accuracy: 0.7701 - val_loss: 0.7139 - val_accuracy: 0.5122
Epoch 4/50
26/26 - 7s - loss: 0.5043 - accuracy: 0.7968 - val_loss: 0.4247 - val_accuracy: 0.9122
Epoch 5/50
26/26 - 7s - loss: 0.4431 - accuracy: 0.8358 - val_loss: 0.3813 - val_accuracy: 0.9024
Epoch 6/50
26/26 - 7s - loss: 0.3874 - accuracy: 0.8552 - val_loss: 0.5415 - val_accuracy: 0.6634
Epoch 7/50
26/26 - 7s - loss: 0.3398 - accuracy: 0.8881 - val_loss: 0.2508 - val_accuracy: 0.9610
Epoch 8/50
26/26 - 7s - loss: 0.2464 - accuracy: 0.9380 - val_loss: 0.1841 - val_accuracy: 0.9707
Epoch 9/50
26/26 - 7s - loss: 0.2074 - accuracy: 0.9440 - val_loss: 0.1569 - val_accuracy: 0.9707
Epoch 10/50
26/26 - 7s - loss: 0.1943 - accuracy: 0.9428 - val_loss: 0.1440 - val_accuracy: 0.9707
Epoch 11/50
26/26 -

In [19]:
results=model.evaluate(validation_batches)

7/7 [==============================] - 0s 60ms/step - loss: 0.1010 - accuracy: 0.9805


## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 8s 295ms/step - loss: 0.6778 - accuracy: 0.5570 - val_loss: 0.6620 - val_accuracy: 0.5122
Epoch 2/5
26/26 [==============================] - 8s 295ms/step - loss: 0.6318 - accuracy: 0.6982 - val_loss: 0.6176 - val_accuracy: 0.6195
Epoch 3/5
26/26 [==============================] - 8s 302ms/step - loss: 0.5742 - accuracy: 0.7281 - val_loss: 0.5348 - val_accuracy: 0.8146
Epoch 4/5
26/26 [==============================] - 7s 286ms/step - loss: 0.5070 - accuracy: 0.7846 - val_loss: 0.4672 - val_accuracy: 0.8537
Epoch 5/5
26/26 [==============================] - 8s 302ms/step - loss: 0.4666 - accuracy: 0.8288 - val_loss: 0.4366 - val_accuracy: 0.8537


In [14]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.608272,0.667445,0.512195,0.661966
1,1,0.699513,0.620532,0.619512,0.617616
2,2,0.726277,0.568708,0.814634,0.534779
3,3,0.753041,0.522784,0.853659,0.467217
4,4,0.815085,0.451332,0.853659,0.436551


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1)])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 8s 287ms/step - loss: 0.6829 - accuracy: 0.5274 - val_loss: 0.6936 - val_accuracy: 0.4341
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 8s 308ms/step - loss: 0.6643 - accuracy: 0.5287 - val_loss: 0.6740 - val_accuracy: 0.4439
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 8s 302ms/step - loss: 0.6555 - accuracy: 0.5530 - val_loss: 0.6678 - val_accuracy: 0.4585
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 7s 283ms/step - loss: 0.6425 - accuracy: 0.5714 - val_loss: 0.6619 - val_accuracy: 0.5220
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 8s 294ms/step - loss: 0.6435 -